## GEM STONE PRICE PREDICTION

#### Problem statement
there are 9 feature with the given feature we need to find the price prediction for the stone. This is a Regression problem statement

In [1]:
# importing the necessary libraries
import numpy as np           # for array manipulation
import pandas as pd                 # for data manipulation
import matplotlib.pyplot as plt     # to graphical representation
%matplotlib inline     
import seaborn as sns           # to visualize the data
import mlflow           # tracking the experiments
from sklearn.preprocessing import OrdinalEncoder,LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score,root_mean_squared_error
from mlflow.models import infer_signature


In [2]:
# reading the data from the local machine using pandas
df=pd.read_csv(r'C:\Users\prasa\Downloads\data.csv')

In [3]:
# reading the first 5 records from the dataset
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [4]:
# basic information about the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193573 entries, 0 to 193572
Data columns (total 11 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       193573 non-null  int64  
 1   carat    193573 non-null  float64
 2   cut      193573 non-null  object 
 3   color    193573 non-null  object 
 4   clarity  193573 non-null  object 
 5   depth    193573 non-null  float64
 6   table    193573 non-null  float64
 7   x        193573 non-null  float64
 8   y        193573 non-null  float64
 9   z        193573 non-null  float64
 10  price    193573 non-null  int64  
dtypes: float64(6), int64(2), object(3)
memory usage: 16.2+ MB


In [5]:
# shape of the data
df.shape

(193573, 11)

In [6]:
# statistical information about the numerical columns in the data
df.describe()

,id,carat,depth,table,x,y,z,price
count,193573.000000,193573.000000,193573.000000,193573.000000,193573.000000,193573.000000,193573.000000,193573.000000
mean,96786.000000,0.790688,61.820574,57.227675,5.715312,5.720094,3.534246,3969.155414
std,55879.856166,0.462688,1.081704,1.918844,1.109422,1.102333,0.688922,4034.374138
min,0.000000,0.200000,52.100000,49.000000,0.000000,0.000000,0.000000,326.000000
25%,48393.000000,0.400000,61.300000,56.000000,4.700000,4.710000,2.900000,951.000000
50%,96786.000000,0.700000,61.900000,57.000000,5.700000,5.720000,3.530000,2401.000000
75%,145179.000000,1.030000,62.400000,58.000000,6.510000,6.510000,4.030000,5408.000000
max,193572.000000,3.500000,71.600000,79.000000,9.650000,10.010000,31.300000,18818.000000


In [7]:
# statistical information about the categorical columns
df.describe(include='O')

,cut,color,clarity
count,193573,193573,193573
unique,5,7,8
top,Ideal,G,SI1
freq,92454,44391,53272


In [8]:
# creating the series that maps the values into segments using cut function and value_counts with 10 segments
def segments(df):
    for col in df.select_dtypes(exclude='O'):
        print(pd.cut(df[col],10).value_counts())
       

segments(df)

id
(-193.572, 19357.2]     19358
(77428.8, 96786.0]      19358
(174214.8, 193572.0]    19358
(19357.2, 38714.4]      19357
(58071.6, 77428.8]      19357
(38714.4, 58071.6]      19357
(96786.0, 116143.2]     19357
(116143.2, 135500.4]    19357
(135500.4, 154857.6]    19357
(154857.6, 174214.8]    19357
Name: count, dtype: int64
carat
(0.197, 0.53]    78019
(0.86, 1.19]     44944
(0.53, 0.86]     36320
(1.19, 1.52]     18441
(1.52, 1.85]      8096
(1.85, 2.18]      6618
(2.18, 2.51]      1039
(2.51, 2.84]        79
(2.84, 3.17]        15
(3.17, 3.5]          2
Name: count, dtype: int64
depth
(61.85, 63.8]     96665
(59.9, 61.85]     82794
(57.95, 59.9]      9471
(63.8, 65.75]      3778
(56.0, 57.95]       406
(65.75, 67.7]       368
(67.7, 69.65]        56
(54.05, 56.0]        24
(52.08, 54.05]        7
(69.65, 71.6]         4
Name: count, dtype: int64
table
(55.0, 58.0]     112421
(58.0, 61.0]      42487
(52.0, 55.0]      34299
(61.0, 64.0]       3968
(64.0, 67.0]        304
(48.97, 52.

In [9]:
#checking the covariance between the column
df.select_dtypes(exclude='O').cov()

,id,carat,depth,table,x,y,z,price
id,3.122558e+09,-19.939165,160.019861,262.484383,-29.491203,-27.626768,1.418993,-3.138048e+05
carat,-1.993917e+01,0.214080,0.013940,0.200415,0.503339,0.499918,0.310556,1.760995e+03
depth,1.600199e+02,0.013940,1.170084,-0.478704,-0.012915,-0.013896,0.060974,8.202188e+00
table,2.624844e+02,0.200415,-0.478704,3.681963,0.505758,0.493475,0.280525,1.354079e+03
x,-2.949120e+01,0.503339,-0.012915,0.505758,1.230818,1.221863,0.756451,4.032737e+03
y,-2.762677e+01,0.499918,-0.013896,0.493475,1.221863,1.215139,0.751468,4.007098e+03
z,1.418993e+00,0.310556,0.060974,0.280525,0.756451,0.751468,0.474614,2.482080e+03
price,-3.138048e+05,1760.995469,8.202188,1354.078697,4032.737140,4007.098146,2482.079924,1.627617e+07


In [10]:
#checking the corelation between the column
df.select_dtypes(exclude='O').corr()

,id,carat,depth,table,x,y,z,price
id,1.000000,-0.000771,0.002647,0.002448,-0.000476,-0.000448,0.000037,-0.001392
carat,-0.000771,1.000000,0.027852,0.225737,0.980563,0.980161,0.974274,0.943396
depth,0.002647,0.027852,1.000000,-0.230631,-0.010762,-0.011653,0.081821,0.001880
table,0.002448,0.225737,-0.230631,1.000000,0.237578,0.233299,0.212208,0.174915
x,-0.000476,0.980563,-0.010762,0.237578,1.000000,0.999109,0.989724,0.901004
y,-0.000448,0.980161,-0.011653,0.233299,0.999109,1.000000,0.989527,0.901033
z,0.000037,0.974274,0.081821,0.212208,0.989724,0.989527,1.000000,0.893037
price,-0.001392,0.943396,0.001880,0.174915,0.901004,0.901033,0.893037,1.000000


As I observed there is corelation with the ID column we can drop the ID column

In [11]:
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [12]:
# value counts for object columns
def val_count(df):
    for col in df.select_dtypes(include='O'):
        print(df[col].value_counts())

val_count(df)

cut
Ideal        92454
Premium      49910
Very Good    37566
Good         11622
Fair          2021
Name: count, dtype: int64
color
G    44391
E    35869
F    34258
H    30799
D    24286
I    17514
J     6456
Name: count, dtype: int64
clarity
SI1     53272
VS2     48027
VS1     30669
SI2     30484
VVS2    15762
VVS1    10628
IF       4219
I1        512
Name: count, dtype: int64


In [13]:
#getting the unique values from the column
df['cut'].unique()

array(['Premium', 'Very Good', 'Ideal', 'Good', 'Fair'], dtype=object)

In [14]:
# ordinal encoding for preserving the order the category
order1 = [['Ideal','Fair','Good','Very Good','Premium']]

ordinalen=OrdinalEncoder(categories=order1)
df['cut']=ordinalen.fit_transform(df[['cut']])
df['cut']=df['cut'].astype(int)


In [15]:
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,4,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,3,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,0,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,0,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,4,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [16]:
#label encoding for without any specific order
def encode(df):
    colum=['clarity','color']
    for col in colum:
            encod=LabelEncoder()
            df[col]=encod.fit_transform(df[col])

    return df.head()

encode(df)

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,4,2,5,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,3,6,3,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,0,3,4,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,0,3,4,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,4,3,5,62.6,59.0,7.65,7.61,4.77,14453


In [17]:
# checking null values
df.isnull().sum()

id         0
carat      0
cut        0
color      0
clarity    0
depth      0
table      0
x          0
y          0
z          0
price      0
dtype: int64

In [18]:
# independent features
X=df.iloc[:,:-1]
X.head()

,id,carat,cut,color,clarity,depth,table,x,y,z
0,0,1.52,4,2,5,62.2,58.0,7.27,7.33,4.55
1,1,2.03,3,6,3,62.0,58.0,8.06,8.12,5.05
2,2,0.70,0,3,4,61.2,57.0,5.69,5.73,3.50
3,3,0.32,0,3,4,61.6,56.0,4.38,4.41,2.71
4,4,1.70,4,3,5,62.6,59.0,7.65,7.61,4.77


In [19]:
# dependent feature
y=df.iloc[:,-1]
y.head()


0    13619
1    13387
2     2772
3      666
4    14453
Name: price, dtype: int64

In [20]:
#splitting the data as train test
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=42)

In [21]:
#shape of the train and test for dependent and independt variables
print('X_train--------'+str(X_train.shape))
print('X_test--------'+str(X_test.shape))
print('y_train--------'+str(y_train.shape))
print('y_test--------'+str(y_test.shape))

X_train--------(145179, 10)
X_test--------(48394, 10)
y_train--------(145179,)
y_test--------(48394,)


In [22]:
# standaring the data using train test split
sc = StandardScaler()
X_train_scaled=sc.fit_transform(X_train)
X_test_scaled=sc.transform(X_test)
print('X_train_scaled',X_train_scaled[:5])
print("-----------------------------------------")
print('X_test_scaled',X_test_scaled[:5])


X_train_scaled [[-1.14291754  0.47201886 -0.9945648   1.52763396 -0.50193043  0.2579731
  -0.64043968  0.60646013  0.62416204  0.64427092]
 [-0.69745758  0.62309937 -0.9945648   0.91152604  0.72283314  0.81246294
  -1.16126779  0.73251523  0.78727755  0.80354857]
 [ 0.53963497  2.65189475  1.28385807  1.52763396  0.72283314 -1.86757127
   1.44287279  2.23617253  2.31875091  2.00537089]
 [-0.32698648 -1.01720328  0.71425235  0.29541811 -1.11431221  1.08970785
  -0.64043968 -1.24835067 -1.23354232 -1.15122266]
 [-1.33478238 -0.11072024  0.14464664 -0.93679775 -0.50193043  1.45936774
   3.52618527  0.03921217 -0.01017603  0.13747838]]
-----------------------------------------
X_test_scaled [[-1.46559688e+00 -6.28710550e-01 -9.94564797e-01 -1.55290568e+00
  -1.11431221e+00  2.57973100e-01 -1.19611559e-01 -6.00067281e-01
  -5.81080301e-01 -5.72031184e-01]
 [ 1.23165816e+00  2.60872889e+00  7.14252352e-01  2.95418106e-01
  -5.01930431e-01 -2.14481619e+00 -1.19611559e-01  2.12812530e+00
   2.

In [23]:
models = [
    ("Linear Regression", LinearRegression()),
    ("Decision Tree", DecisionTreeRegressor(random_state=42)),
    ("SVM", SVR()),
    ("Random Forest", RandomForestRegressor(random_state=42)),
    ]

In [24]:
best_model = None
best_r2_score = 0.0
best_model_name = ""

# model traking using mlflow
mlflow.set_tracking_uri(' http://127.0.0.1:5000')           #setting the tracking uri
mlflow.set_experiment('GEM STONE')                     # setting the experiment name

with mlflow.start_run(nested=True):
    #inferencing the model
    

    for model_name, model in models:
        # Train the model
        model.fit(X_train_scaled, y_train)
        
        # Make predictions on the test set
        y_pred = model.predict(X_test_scaled)
        y_pred_train = model.predict(X_train_scaled)
        
        # Evaluate the model's accuracy
        mae = mean_absolute_error(y_test, y_pred)
        mse=mean_squared_error(y_test,y_pred)
        rmse=root_mean_squared_error(y_test,y_pred)
        r2=r2_score(y_test,y_pred)
        print(f'mae of the {model_name}'+'------>'+str(mae))
        print(f'mse of {model_name}'+'------>'+str(mse))
        print(f'rmse of {model_name}'+'------>'+str(rmse))
        print(f'r2_score of {model_name}'+'------>'+str(r2))
        #mlflow.log_param("model", model_name)
        #mlflow.log_metric("accuracy", accuracy)
        signature=infer_signature(X_train_scaled,y_pred_train)
        # Log the model itself
        model_info=mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path='Best model',
        signature=signature,
        #registered_model_name='BEST MODEL'
        input_example=X_train_scaled
    )

        # Check if this is the best model so far
        if r2 > best_r2_score:
            best_r2_score = r2
            best_model = model
            best_model_name = model_name

    # Log the best model as the final model
    mlflow.log_param("best_model", best_model_name)
    mlflow.log_metric("best_accuracy", best_r2_score)

    print(f"The best model is: {best_model_name} with accuracy: {best_r2_score}")
    # logging the model
    

2025/02/07 12:33:29 INFO mlflow.tracking.fluent: Experiment with name 'GEM STONE' does not exist. Creating a new experiment.


mae of the Linear Regression------>711.1127062058574
mse of Linear Regression------>1237657.2285300554
rmse of Linear Regression------>1112.500439788702
r2_score of Linear Regression------>0.9237397552211855
mae of the Decision Tree------>424.25773856263174
mse of Decision Tree------>702574.4846261933
rmse of Decision Tree------>838.1971633369999
r2_score of Decision Tree------>0.9567097408411073
mae of the SVM------>948.9744416688126
mse of SVM------>4068111.416907932
rmse of SVM------>2016.9559779300916
r2_score of SVM------>0.7493367587653086
mae of the Random Forest------>306.17068190271516
mse of Random Forest------>371973.0811886928
rmse of Random Forest------>609.8959593149416
r2_score of Random Forest------>0.9770802791203584
🏃 View run angry-cod-984 at:  http://127.0.0.1:5000/#/experiments/527744177238912881/runs/de6dd5b6f70d46abba67f3af047bf16a
🧪 View experiment at:  http://127.0.0.1:5000/#/experiments/527744177238912881


MlflowException: API request to http://127.0.0.1:5000/api/2.0/mlflow-artifacts/artifacts/527744177238912881/de6dd5b6f70d46abba67f3af047bf16a/artifacts/Best model/model.pkl failed with exception HTTPConnectionPool(host='127.0.0.1', port=5000): Max retries exceeded with url: /api/2.0/mlflow-artifacts/artifacts/527744177238912881/de6dd5b6f70d46abba67f3af047bf16a/artifacts/Best%20model/model.pkl (Caused by ProtocolError('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)))